In [31]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from shapely import wkt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

### **READ IN DATA**

In [2]:
train = pd.read_csv('data\Train.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_11328\1100633172.py:1: SyntaxWarning: invalid escape sequence '\T'
  train = pd.read_csv('data\Train.csv')


In [3]:
train.head(3)

,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,54,Pea,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,POLYGON ((78.18143246076087 17.978884603571068...
1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,58,Pea,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,POLYGON ((78.17545177547453 17.981074445477304...
2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,91,Pea,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,POLYGON ((78.16914224770792 17.976212849967755...


In [4]:
test = pd.read_csv('data\Test.csv')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Aduragbemi\AppData\Local\Temp\ipykernel_11328\3210526593.py:1: SyntaxWarning: invalid escape sequence '\T'
  test = pd.read_csv('data\Test.csv')


In [5]:
test.head(3)

,FarmID,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
0,85197,Paddy,Telangana,Medak,Nizampet,2023-11-12 0:00:00,2024-05-14 0:00:00,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,POLYGON ((78.54742059207287 18.062307326758276...
1,779677,Paddy,Telangana,Medak,Nizampet,2023-12-20 0:00:00,2024-05-28 0:00:00,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,"POLYGON ((78.54812514990353 18.06739449795583,..."
2,1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29 0:00:00,2024-05-18 0:00:00,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,POLYGON ((78.56002792938853 18.079607077817514...


In [6]:
test.shape

(2718, 19)

In [7]:
test.columns

Index(['FarmID', 'Crop', 'State', 'District', 'Sub-District', 'SDate', 'HDate',
       'CropCoveredArea', 'CHeight', 'CNext', 'CLast', 'CTransp', 'IrriType',
       'IrriSource', 'IrriCount', 'WaterCov', 'ExpYield', 'Season',
       'geometry'],
      dtype='object')

In [8]:
train.shape

(7888, 20)

In [9]:
train.columns

Index(['FarmID', 'category', 'Crop', 'State', 'District', 'Sub-District',
       'SDate', 'HDate', 'CropCoveredArea', 'CHeight', 'CNext', 'CLast',
       'CTransp', 'IrriType', 'IrriSource', 'IrriCount', 'WaterCov',
       'ExpYield', 'Season', 'geometry'],
      dtype='object')

### **DATA WRANGLING**

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7888 entries, 0 to 7887
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FarmID           7888 non-null   int64 
 1   category         7888 non-null   object
 2   Crop             7888 non-null   object
 3   State            7888 non-null   object
 4   District         7888 non-null   object
 5   Sub-District     7888 non-null   object
 6   SDate            7888 non-null   object
 7   HDate            7888 non-null   object
 8   CropCoveredArea  7888 non-null   int64 
 9   CHeight          7888 non-null   int64 
 10  CNext            7888 non-null   object
 11  CLast            7888 non-null   object
 12  CTransp          7888 non-null   object
 13  IrriType         7888 non-null   object
 14  IrriSource       7888 non-null   object
 15  IrriCount        7888 non-null   int64 
 16  WaterCov         7888 non-null   int64 
 17  ExpYield         7888 non-null   

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   FarmID           2718 non-null   int64 
 1   Crop             2718 non-null   object
 2   State            2718 non-null   object
 3   District         2718 non-null   object
 4   Sub-District     2718 non-null   object
 5   SDate            2718 non-null   object
 6   HDate            2718 non-null   object
 7   CropCoveredArea  2718 non-null   int64 
 8   CHeight          2718 non-null   int64 
 9   CNext            2718 non-null   object
 10  CLast            2718 non-null   object
 11  CTransp          2718 non-null   object
 12  IrriType         2718 non-null   object
 13  IrriSource       2718 non-null   object
 14  IrriCount        2718 non-null   int64 
 15  WaterCov         2718 non-null   int64 
 16  ExpYield         2718 non-null   int64 
 17  Season           2718 non-null   

### **SET _FarmID_ AS THE INDEX**

In [12]:
train.set_index('FarmID', inplace=True)

In [13]:
train.head(3)

,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
FarmID,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,54,Pea,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,POLYGON ((78.18143246076087 17.978884603571068...
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,58,Pea,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,POLYGON ((78.17545177547453 17.981074445477304...
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,91,Pea,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,POLYGON ((78.16914224770792 17.976212849967755...


In [14]:
test.set_index('FarmID', inplace=True)

In [15]:
test.head(3)

,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,geometry
FarmID,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,2023-11-12 0:00:00,2024-05-14 0:00:00,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,POLYGON ((78.54742059207287 18.062307326758276...
779677,Paddy,Telangana,Medak,Nizampet,2023-12-20 0:00:00,2024-05-28 0:00:00,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,"POLYGON ((78.54812514990353 18.06739449795583,..."
1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29 0:00:00,2024-05-18 0:00:00,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,POLYGON ((78.56002792938853 18.079607077817514...


### **CONVERT *SDate* AND *HDate* FEATURE TO DATETIME**

In [16]:
train['SDate'] = pd.to_datetime(train['SDate'])
train['HDate'] = pd.to_datetime(train['HDate']) 

print(train['SDate'].dtype)
print(train['HDate'].dtype)

datetime64[ns]
datetime64[ns]


In [17]:
test['SDate'] = pd.to_datetime(test['SDate'])
test['HDate'] = pd.to_datetime(test['HDate'])

print(test['SDate'].dtype)
print(test['HDate'].dtype)

datetime64[ns]
datetime64[ns]


### **EXTRACTING FEATURES FROM _geometry_**

In [18]:
train['geometry'] = train['geometry'].astype(str).apply(wkt.loads)  
train['Area'] = train['geometry'].apply(lambda x: x.area if x else None)
train['Perimeter'] = train['geometry'].apply(lambda x: x.length if x else None)   

train = train.drop(['geometry'], axis=1)

In [19]:
train.head(3)

,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,Area,Perimeter
FarmID,,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25,2024-04-14,97,54,Pea,Lentil,Transplanting,Flood,Groundwater,4,87,17,Rabi,3.982611e-08,0.000849
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13,2024-04-26,82,58,Pea,Lentil,Transplanting,Flood,Canal,5,94,15,Rabi,5.989901e-08,0.001016
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19,2024-04-28,92,91,Pea,Lentil,Transplanting,Flood,Canal,3,99,20,Rabi,3.867260e-08,0.000798


In [20]:
test['geometry'] = test['geometry'].astype(str).apply(wkt.loads)  
test['Area'] = test['geometry'].apply(lambda x: x.area if x else None)
test['Perimeter'] = test['geometry'].apply(lambda x: x.length if x else None)   

test = test.drop(['geometry'], axis=1)

In [21]:
test.head(3)

,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,IrriSource,IrriCount,WaterCov,ExpYield,Season,Area,Perimeter
FarmID,,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,2023-11-12,2024-05-14,81,99,Lentil,Pea,Transplanting,Flood,Groundwater,5,81,19,Rabi,1.011755e-07,0.001353
779677,Paddy,Telangana,Medak,Nizampet,2023-12-20,2024-05-28,91,63,Lentil,Pea,Transplanting,Flood,Canal,5,97,15,Rabi,1.679801e-07,0.002157
1331840,Paddy,Telangana,Medak,Nizampet,2023-11-29,2024-05-18,94,89,Pea,Lentil,Transplanting,Flood,Canal,5,88,15,Rabi,7.013044e-08,0.001091


### **EXTRACT *year*, *month* AND *day* FROM *SDate* AND *HDate***

In [22]:
train['SYear'] = train['SDate'].dt.year
train['SMonth'] = train['SDate'].dt.month
train['SDay'] = train['SDate'].dt.day

train = train.drop(['SDate'], axis=1)

train['HYear'] = train['HDate'].dt.year
train['HMonth'] = train['HDate'].dt.month
train['HDay'] = train['HDate'].dt.day

train = train.drop(['HDate'], axis=1)

train.head(3)

,category,Crop,State,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,...,ExpYield,Season,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
FarmID,,,,,,,,,,,,,,,,,,,,,
1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,97,54,Pea,Lentil,Transplanting,...,17,Rabi,3.982611e-08,0.000849,2023,11,25,2024,4,14
1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,82,58,Pea,Lentil,Transplanting,...,15,Rabi,5.989901e-08,0.001016,2023,11,13,2024,4,26
1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,92,91,Pea,Lentil,Transplanting,...,20,Rabi,3.867260e-08,0.000798,2023,12,19,2024,4,28


In [23]:
test['SYear'] = test['SDate'].dt.year
test['SMonth'] = test['SDate'].dt.month
test['SDay'] = test['SDate'].dt.day

test = test.drop(['SDate'], axis=1)

test['HYear'] = test['HDate'].dt.year
test['HMonth'] = test['HDate'].dt.month
test['HDay'] = test['HDate'].dt.day

test = test.drop(['HDate'], axis=1)

test.head(3)

,Crop,State,District,Sub-District,CropCoveredArea,CHeight,CNext,CLast,CTransp,IrriType,...,ExpYield,Season,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
FarmID,,,,,,,,,,,,,,,,,,,,,
85197,Paddy,Telangana,Medak,Nizampet,81,99,Lentil,Pea,Transplanting,Flood,...,19,Rabi,1.011755e-07,0.001353,2023,11,12,2024,5,14
779677,Paddy,Telangana,Medak,Nizampet,91,63,Lentil,Pea,Transplanting,Flood,...,15,Rabi,1.679801e-07,0.002157,2023,12,20,2024,5,28
1331840,Paddy,Telangana,Medak,Nizampet,94,89,Pea,Lentil,Transplanting,Flood,...,15,Rabi,7.013044e-08,0.001091,2023,11,29,2024,5,18


### **DROP *State* FEATURE**

In [24]:
train = train.drop(['State'], axis=1)
test = test.drop(['State'], axis=1)

In [25]:
train.describe()

,CropCoveredArea,CHeight,IrriCount,WaterCov,ExpYield,Area,Perimeter,SYear,SMonth,SDay,HYear,HMonth,HDay
count,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000,7.888000e+03,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000,7888.000000
mean,77.419244,87.448783,3.538159,57.154285,15.694473,3.614335e-07,0.002310,2023.015340,8.439528,16.619549,2023.781694,5.626395,15.225279
std,14.479677,50.308776,1.405152,22.860627,5.646621,3.968992e-07,0.001213,0.122908,3.299687,7.479800,0.413123,3.557494,9.201316
min,40.000000,20.000000,1.000000,20.000000,5.000000,4.582125e-09,0.000274,2023.000000,1.000000,1.000000,2023.000000,1.000000,1.000000
25%,66.000000,54.000000,2.000000,41.000000,11.000000,1.065698e-07,0.001395,2023.000000,6.000000,11.000000,2024.000000,3.000000,5.000000
50%,79.000000,76.000000,4.000000,51.000000,15.000000,2.369511e-07,0.002100,2023.000000,10.000000,16.000000,2024.000000,4.000000,16.000000
75%,89.000000,100.000000,5.000000,82.000000,20.000000,4.722745e-07,0.003003,2023.000000,11.000000,23.000000,2024.000000,9.000000,23.000000
max,100.000000,250.000000,6.000000,100.000000,30.000000,5.949748e-06,0.010898,2024.000000,12.000000,31.000000,2024.000000,12.000000,31.000000


In [26]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7888 entries, 1326576 to 1330504
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   category         7888 non-null   object 
 1   Crop             7888 non-null   object 
 2   District         7888 non-null   object 
 3   Sub-District     7888 non-null   object 
 4   CropCoveredArea  7888 non-null   int64  
 5   CHeight          7888 non-null   int64  
 6   CNext            7888 non-null   object 
 7   CLast            7888 non-null   object 
 8   CTransp          7888 non-null   object 
 9   IrriType         7888 non-null   object 
 10  IrriSource       7888 non-null   object 
 11  IrriCount        7888 non-null   int64  
 12  WaterCov         7888 non-null   int64  
 13  ExpYield         7888 non-null   int64  
 14  Season           7888 non-null   object 
 15  Area             7888 non-null   float64
 16  Perimeter        7888 non-null   float64
 17  SYear     